In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [5]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of species : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [6]:
# Separate known and unknown samples
df_known = df[df["ScientificName"] != "Unknown"]
df_unknown = df[df["ScientificName"] == "Unknown"]

# Extract features for known samples (passing both img and taxa)
X_known = np.vstack([
    extract_features(img, taxa) for img, taxa in tqdm(zip(df_known["ImageFilePath"], df_known["ScientificName"]), desc="Processing ...")
])

# Encode labels only for known species
le = LabelEncoder()
y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index

# Train-test split only on known species
X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)

# Extract features for unknown samples (passing img but with a placeholder for taxa)
X_test_unknown = np.vstack([
    extract_features(img, "Unknown") for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")
])
test_idx_unknown = df_unknown.index

# Assign a placeholder label (-1) for unknown samples
y_test_unknown = np.full(len(X_test_unknown), -1)

# Combine known and unknown test sets
X_test = np.vstack([X_test_known, X_test_unknown])
y_test = np.concatenate([y_test_known, y_test_unknown])
test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")


Processing ...: 0it [00:00, ?it/s]

Processing Unknown ...:   0%|          | 0/4328 [00:00<?, ?it/s]

Train set: (16222, 768), Test set: (8384, 768)


In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 19.12% | Prec: 16.11% | Rec: 19.12% | F1: 15.42% | Bal Acc: 22.56% | MCC: 0.2227
LogisticRegression        | Acc: 36.95% | Prec: 29.24% | Rec: 36.95% | F1: 30.78% | Bal Acc: 54.57% | MCC: 0.4316


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NearestNeighbor           | Acc: 18.76% | Prec: 14.47% | Rec: 18.76% | F1: 14.19% | Bal Acc: 20.42% | MCC: 0.2181
MLP_Baseline              | Acc: 33.10% | Prec: 27.43% | Rec: 33.10% | F1: 28.33% | Bal Acc: 46.83% | MCC: 0.3866


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Cicindela tranquebarica,Cicindela willistoni,Cicindela amargosae,Cicindela willistoni
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.191198,0.161071,0.191198,0.154244,0.225569,0.222733
LogisticRegression,LogisticRegression,0.369513,0.292385,0.369513,0.307821,0.545733,0.431565
NearestNeighbor,NearestNeighbor,0.187619,0.144694,0.187619,0.141946,0.20423,0.218115
MLP_Baseline,MLP_Baseline,0.330988,0.274344,0.330988,0.283335,0.468295,0.386631


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M/6.ViLT-Insect1M-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M/6.ViLT-linear-Insect1M-metrics.csv", index=False)